# Método
Para poder usar los datos de las noticias, se usará sólo el contenido de la noticia. Luego la fecha será usada para hacer coincidir el cambio del precio con la noticia. <br>
En el archivo de noticias, se tienen varias del mismo día, por lo que las que compartan fecha se mezclaran en una sóla. <br>
Por otra parte, el archivo del Bitcoin será usado para ver si ese día hubo un aumento o decremento en su valor.

## Archivo de noticias

In [109]:
import pandas as pd
a_1 = pd.read_csv('faltante.csv')
a_2 = pd.read_csv('news_data.csv')

In [110]:
print(a_1.shape, a_2.shape)

(15, 5) (1102, 5)


In [111]:
a_1 = a_1.drop('i', axis=1)
a_2 = a_2.drop('i', axis=1)
data = pd.concat([a_1, a_2])
data.shape

(1117, 4)

In [112]:
data.columns

Index(['Enlace', 'Titulo', 'Cuerpo', 'Fecha'], dtype='object')

In [113]:
data = data.drop(['Enlace', 'Titulo'], axis=1)
data.head()

,Cuerpo,Fecha
0,"Panamá fue sede de actualización, discusiones ...",12.07.2022
1,Arijit Sarkar\n09 Jul 2022\nLa baja en el prec...,10.07.2022
2,Yashu Gola\n06 Jul 2022\nLas acciones mineras ...,06.07.2022
3,William Suberg\n30 Jun 2022\nEl precio de bitc...,30.06.2022
4,Zhiyuan Sun\n29 Jun 2022\nLa empresa de minerí...,29.06.2022


### Cambiando formato de fecha

In [114]:
def format(fecha):
    # Ejemplo:
    #   12.07.2022 -> 2022-07-12
    a = fecha.split('.')
    if len(a)>1:
        return '{}-{}-{}'.format(a[-1],a[1],a[0])
    else:
        # La fecha de hoy, en caso de encontrarse un 'Hace'
        return '2022-07-16'

In [115]:
data['Fecha'] = data['Fecha'].apply(format)
data.head()

,Cuerpo,Fecha
0,"Panamá fue sede de actualización, discusiones ...",2022-07-12
1,Arijit Sarkar\n09 Jul 2022\nLa baja en el prec...,2022-07-10
2,Yashu Gola\n06 Jul 2022\nLas acciones mineras ...,2022-07-06
3,William Suberg\n30 Jun 2022\nEl precio de bitc...,2022-06-30
4,Zhiyuan Sun\n29 Jun 2022\nLa empresa de minerí...,2022-06-29


### Se juntarán las notas con la misma fecha

In [116]:
aux = data.groupby('Fecha')

In [117]:
notas, fechas = [], []
for fecha in data['Fecha'].unique():
    nota = ''
    for text in aux.get_group(fecha)['Cuerpo']:
        nota += str(text)
    notas.append(nota)
    fechas.append(fecha)
data_aux = pd.DataFrame({'Texto': notas, 'Fecha': fechas})
data_aux.head()

,Texto,Fecha
0,"Panamá fue sede de actualización, discusiones ...",2022-07-12
1,Arijit Sarkar\n09 Jul 2022\nLa baja en el prec...,2022-07-10
2,Yashu Gola\n06 Jul 2022\nLas acciones mineras ...,2022-07-06
3,William Suberg\n30 Jun 2022\nEl precio de bitc...,2022-06-30
4,Zhiyuan Sun\n29 Jun 2022\nLa empresa de minerí...,2022-06-29


## Archivo de Bitcoin

In [118]:
prices = pd.read_csv('BTC-USD.csv')
prices.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-04-01,45554.164063,46616.242188,44403.140625,46281.644531,46281.644531,3.816264e+10
1,2022-04-02,46285.500000,47028.281250,45782.511719,45868.949219,45868.949219,2.933659e+10
2,2022-04-03,45859.128906,47313.476563,45634.105469,46453.566406,46453.566406,2.541440e+10
3,2022-04-04,46445.273438,46791.089844,45235.816406,46622.675781,46622.675781,3.249979e+10
4,2022-04-05,46624.507813,47106.140625,45544.808594,45555.992188,45555.992188,2.964060e+10


In [119]:
prices = prices.drop(['High', 'Low', 'Adj Close', 'Volume'], axis=1)
prices['class'] = 0
prices.head()

,Date,Open,Close,class
0,2022-04-01,45554.164063,46281.644531,0
1,2022-04-02,46285.500000,45868.949219,0
2,2022-04-03,45859.128906,46453.566406,0
3,2022-04-04,46445.273438,46622.675781,0
4,2022-04-05,46624.507813,45555.992188,0


In [120]:
def cambio(open, close):
    if close - open >= 0:
        return 1
    else:
        return 0

### Generando clasificación

In [121]:
prices['class'] = prices.apply(lambda x: cambio(x['Open'], x['Close']), axis=1)
prices = prices.drop(['Open','Close'],axis=1)
prices.head()

,Date,class
0,2022-04-01,1
1,2022-04-02,0
2,2022-04-03,1
3,2022-04-04,1
4,2022-04-05,0


In [122]:
print(prices.shape, data_aux.shape)

(107, 2) (107, 2)


## Se juntan ambas tablas mediante las fechas

In [123]:
data_aux['class'] = data_aux['Fecha'].map(prices.set_index('Date')['class'])
data_aux = data_aux.drop(['Fecha'],axis=1)
data_aux.head()

,Texto,class
0,"Panamá fue sede de actualización, discusiones ...",0
1,Arijit Sarkar\n09 Jul 2022\nLa baja en el prec...,0
2,Yashu Gola\n06 Jul 2022\nLas acciones mineras ...,1
3,William Suberg\n30 Jun 2022\nEl precio de bitc...,0
4,Zhiyuan Sun\n29 Jun 2022\nLa empresa de minerí...,0


In [127]:
data_aux.to_csv('Datos.csv')